In [21]:
# need to match the system-wide installed cuda-11 for deepspeed to compile
# so install the matching pytorch

# pt-1.8 doesn't have cu110 build at the moment. 
# colab will eventually upgrade to cuda-11.1 and then we can use 1.8.0+cu111
# 
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [24]:
# either install the release
#!pip install deepspeed
# or the master 
!pip install git+https://github.com/microsoft/deepspeed

  Cloning https://github.com/microsoft/deepspeed to /tmp/pip-req-build-ocznqahs
  Running command git clone -q https://github.com/microsoft/deepspeed /tmp/pip-req-build-ocznqahs
  Running command git submodule update --init --recursive -q
  Created wheel for deepspeed: filename=deepspeed-0.3.11+564eb4b-cp37-none-any.whl size=335679 sha256=2765352eeeb8f1e699a4d85ff8253211c041cf0e6e72f85bf5837ad49815a8c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-p36m30ku/wheels/33/7c/6d/1ac44092dd4e4b5ddd1dec9474fed46ec3fe5588be7b6ffe9e
Successfully built deepspeed


In [25]:
%%bash
git clone https://github.com/huggingface/transformers
cd transformers
# examples change a lot so let's pick a sha that we know this notebook will work with
# comment out/remove the next line if you want the master
git checkout 1aa9c13f70ae75be7fd6
pip install -e .
pip install -r examples/_tests_requirements.txt

# if needed free up some space used by cached pip packages
# rm -rf /root/.cache/pip


Obtaining file:///content/transformers
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Found existing installation: transformers 4.4.0.dev0
    Can't uninstall 'transformers'. No files were found to uninstall.
  Running setup.py develop for transformers
Processing /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f/seqeval-1.2.2-cp37-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7e/57/0c7ca4e31a126189dab99c19951910bd081dea5bbd25f24b77107750eae7/sacrebleu-1.5.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1f/56/a81022436c08b9405a5247b71635394d44fe7e1dbedc4b28c740e09c2840/rouge_score-0.0.4-py2.py3-none-any.whl
  Using cached https://files.pythonho

fatal: destination path 'transformers' already exists and is not an empty directory.
HEAD is now at 1aa9c13f7 Fix GPU tests with speech
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.16 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.21.0 which is incompatible.


In [26]:
%%bash

cd transformers

cat <<'EOT' > ds_config.json
{
    "fp16": {
        "enabled": true,
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 16,        
        "hysteresis": 2,
        "min_loss_scale": 1
    },

    "zero_optimization": {
        "stage": 2,
       "allgather_partitions": true,
       "allgather_bucket_size": 2e8,
       "reduce_scatter": true,
       "reduce_bucket_size": 2e8,
        "overlap_comm": true,
        "contiguous_gradients": true,
        "cpu_offload": true
    },

    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": 3e-5,
            "betas": [ 0.9, 0.999 ],
            "eps": 1e-8,
            "weight_decay": 3e-7
        }
    },

    "scheduler": {
        "type": "WarmupLR",
        "params": {
            "warmup_min_lr": 0,
            "warmup_max_lr": 3e-5,
            "warmup_num_steps": 500
        }
    },
    "steps_per_print": 2000,
    "wall_clock_breakdown": false
}
EOT


In [27]:
#!ls -l transformers
#!cat transformers/ds_config.json

In [28]:
!cd transformers; export BS=16; rm -r output_dir; PYTHONPATH=src USE_TF=0 CUDA_VISIBLE_DEVICES=0 deepspeed --num_gpus=1 examples/seq2seq/run_seq2seq.py --model_name_or_path google/mt5-small \
--output_dir output_dir --adam_eps 1e-06  --evaluation_strategy=steps  --do_train --label_smoothing 0.1 --learning_rate 3e-5 --logging_first_step --logging_steps 1000 \
--max_source_length 128 --max_target_length 128 --num_train_epochs 1 --overwrite_output_dir  --per_device_train_batch_size $BS --predict_with_generate --sortish_sampler \
--val_max_target_length 128 --warmup_steps 500 --max_train_samples 2000 --max_val_samples 500 \
--task translation_en_to_ro  --dataset_name wmt16 --dataset_config ro-en --source_prefix "translate English to Romanian: " --deepspeed  ds_config.json --fp16


rm: cannot remove 'output_dir': No such file or directory
[2021-03-11 00:35:09,293] [WARNING] [runner.py:117:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2021-03-11 00:35:09,310] [INFO] [runner.py:358:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 examples/seq2seq/run_seq2seq.py --model_name_or_path google/mt5-small --output_dir output_dir --adam_eps 1e-06 --evaluation_strategy=steps --do_train --label_smoothing 0.1 --learning_rate 3e-5 --logging_first_step --logging_steps 1000 --max_source_length 128 --max_target_length 128 --num_train_epochs 1 --overwrite_output_dir --per_device_train_batch_size 16 --predict_with_generate --sortish_sampler --val_max_target_length 128 --warmup_steps 500 --max_train_samples 2000 --max_val_samples 500 --task translation_en_to_ro --dataset_name wmt16 --dataset_config ro-en --source_prefix translate English to 